In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [4]:
df = pd.read_csv("../data/processed/market_data_processed.csv")
df.head()

,Date,SP500_Close,VIX_Close,SP500_LogReturn
0,2010-01-05,1136.520020,19.350000,0.003111
1,2010-01-06,1137.140015,19.160000,0.000545
2,2010-01-07,1141.689941,19.059999,0.003993
3,2010-01-08,1144.979980,18.129999,0.002878
4,2010-01-11,1146.979980,17.549999,0.001745


In [5]:
df["Date"] = pd.to_datetime(df["Date"])
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 4058 entries, 0 to 4057
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             4058 non-null   datetime64[us]
 1   SP500_Close      4058 non-null   float64       
 2   VIX_Close        4058 non-null   float64       
 3   SP500_LogReturn  4058 non-null   float64       
dtypes: datetime64[us](1), float64(3)
memory usage: 126.9 KB


In [7]:
returns = df["SP500_LogReturn"].dropna().values
returns

array([ 0.00311083,  0.00054537,  0.00399322, ..., -0.00282612,
        0.00691576, -0.01043996], shape=(4058,))

In [8]:
T = len(returns)
T

4058

# Variance Recursion Function

In [9]:
def garch_variance(params, returns):
    omega, alpha, beta = params

    T = len(returns)
    sigma2 = np.zeros(T)

    # Initialize with unconditional variance
    sigma2[0] = np.var(returns)

    for t in range(1, T):
        sigma2[t] = (
            omega
            + alpha * returns[t-1]**2
            + beta * sigma2[t-1]
        )

    return sigma2

# Log-Likelihood Function

In [10]:
def garch_log_likelihood(params, returns):
    omega, alpha, beta = params

    # Enforce positivity & stationarity
    if omega <= 0 or alpha < 0 or beta < 0 or (alpha + beta) >= 1:
        return 1e10

    sigma2 = garch_variance(params, returns)

    loglik = -0.5 * np.sum(
        np.log(2 * np.pi)
        + np.log(sigma2)
        + (returns**2) / sigma2
    )

    return -loglik  # negative for minimization

# Parameter Estimation via MLE

In [11]:
initial_guess = [0.000001, 0.05, 0.9]

bounds = [
    (1e-8, None),   # omega > 0
    (0, 1),         # alpha
    (0, 1)          # beta
]

result = minimize(
    garch_log_likelihood,
    initial_guess,
    args=(returns,),
    bounds=bounds,
    method="L-BFGS-B"
)

omega_hat, alpha_hat, beta_hat = result.x

omega_hat, alpha_hat, beta_hat

(np.float64(3.2513175183929927e-06),
 np.float64(0.05000000041925546),
 np.float64(0.9000000000935806))